# ２章　小売店のデータでデータ加工を行う１０本ノック

本章では、ある小売店の売上履歴と顧客台帳データを用いて、データ分析の素地となる「データの加工」を習得することが目的です。
実際の現場データは手入力のExcel等、決して綺麗なデータではない事が多いため、
データの揺れや整合性の担保など、汚いデータを取り扱うデータ加工を主体に進めて行きます。

### ノック１１：データを読み込んでみよう

In [2]:
import pandas as pd
uriage_df = pd.read_csv('uriage.csv')
uriage_df

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二
...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500.0,福島友也
2995,2019-06-22 04:03:43,商品M,1300.0,大倉晃司
2996,2019-03-29 11:14:05,商品Q,NaN,尾形小雁
2997,2019-07-14 12:56:49,商品H,NaN,芦田博之


In [3]:
user_df = pd.read_excel('kokyaku_daicho.xlsx')
user_df

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127
...,...,...,...,...,...
195,川上 りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017/06/20
196,小松 季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018/06/20
197,白鳥 りえ,しらとり りえ,F市,shiratori_rie@example.com,2017/04/29
198,大西 隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019/04/19


### ノック１２：データの揺れを見てみよう

In [ ]:
# 入力データに欠損値や、全角半角などの表記ゆれが存在しているかをチェックする。

In [4]:
uriage_df['item_name'] #全角半角が混じってる。小文字と大文字が混じっている

0         商品A
1       商 品 S
2       商 品 a
3         商品Z
4         商品a
        ...  
2994      商品Y
2995      商品M
2996      商品Q
2997      商品H
2998      商品D
Name: item_name, Length: 2999, dtype: object

In [5]:
uriage_df['item_price'] # 欠損値が含まれている

0        100.0
1          NaN
2          NaN
3       2600.0
4          NaN
         ...  
2994    2500.0
2995    1300.0
2996       NaN
2997       NaN
2998     400.0
Name: item_price, Length: 2999, dtype: float64

### ノック１３：データに揺れがあるまま集計しよう
欠損値の影響を見るために、まずはデータに揺れを含んだまま各種値を集計してみる。

In [6]:
# 商品ごとの月別売上個数
uriage_df['purchase_date'] = pd.to_datetime(uriage_df['purchase_date'])
uriage_df['purchase_month'] = uriage_df['purchase_date'].dt.strftime("%Y%m")
uriage_df

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二,201907
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二,201904
...,...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500.0,福島友也,201902
2995,2019-06-22 04:03:43,商品M,1300.0,大倉晃司,201906
2996,2019-03-29 11:14:05,商品Q,NaN,尾形小雁,201903
2997,2019-07-14 12:56:49,商品H,NaN,芦田博之,201907


In [7]:
pivot = uriage_df.pivot_table(index='purchase_month', columns='item_name', aggfunc="size", fill_value=0)
pivot
# 本来は26商品しかないが、99商品として集計されてしまっている

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


In [8]:
# 各月の売上金額も見てみる
pivot = uriage_df.pivot_table(index='purchase_month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
pivot

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,...,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,0,1400,0,0,0,0,0,0,0,0,...,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,2400,0,0,...,0,0,0,0,0,1900,2000,2200,0,0
201903,0,0,500,1300,1600,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,2300,0,0,0,0,0,0,0,0,1700,...,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,0,1900,0,0,0,0,...,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,0,2300,0,0,0,...,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,...,0,0,1500,0,1800,0,0,0,0,0


### ノック１４：商品名の揺れを補正しよう

対応方針 : 商品名のデータを見てから、全角に揃える

In [9]:
print(len(uriage_df['item_name'].unique()))

99


In [10]:
# 僕のやり方 : 空白を除去してから、大文字に揃える
uriage_df['item_name'] = uriage_df['item_name'].str.replace(' ', '')
uriage_df['item_name'] = uriage_df['item_name'].str.replace('　', '')
uriage_df['item_name'] = uriage_df['item_name'].str.upper()

In [11]:
print(len(uriage_df['item_name'].unique()))
print(uriage_df['item_name'].unique())

26
['商品A' '商品S' '商品Z' '商品V' '商品O' '商品U' '商品L' '商品C' '商品I' '商品R' '商品X' '商品G'
 '商品P' '商品Q' '商品Y' '商品N' '商品W' '商品E' '商品K' '商品B' '商品F' '商品D' '商品M' '商品H'
 '商品T' '商品J']


### ノック１５：金額欠損値の補完をしよう

In [ ]:
# 次は金額の欠損値を行う。
# 業務で行う場合、欠損値はヒアリング等を行うことが必要になるかもしれない。
# 今回は、普通に欠損しているだけなので、コードで埋める

In [14]:
# nullチェック
uriage_df.isnull().any()

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

In [27]:
# item_priceが欠損しているが、商品名が存在するので、そこから補完する
null_flg = uriage_df["item_price"].isnull() # nullにフラグを立てる
# print(null_flg)

for trg in list(uriage_df.loc[null_flg, "item_name"].unique()): # 欠損がある商品名をすべて取得する
    hoten_price = uriage_df.loc[(~null_flg) & (uriage_df["item_name"] == trg), "item_price"].max() # 欠損してない要素かつ、対象の商品を取得. 一つだけを選択するためにmaxを指定
    print(hoten_price)
    uriage_df["item_price"].loc[(null_flg) & (uriage_df["item_name"] == trg)] = hoten_price

1900.0
100.0
1600.0
1400.0
2300.0
1800.0
900.0
1200.0
600.0
1500.0
200.0
300.0
2200.0
1700.0
2100.0
1100.0
2000.0
2400.0
500.0
1300.0
700.0
1000.0
400.0
800.0
2500.0


/home/mike/www/privates/100_knock/python_data_analyze/.venv/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [28]:
# nullチェック
uriage_df.isnull().any()

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

In [33]:
# ただしく値を置換できたからをチェックする。
# そのために、各種商品名の最小値、最大値を確認する
for item_name in list(uriage_df["item_name"].unique()):
    max_price = uriage_df['item_price'].loc[uriage_df['item_name'] == item_name].max()
    min_price = uriage_df['item_price'].loc[uriage_df['item_name'] == item_name].min()
    print_text = "item_name : {0}, max_price: {1}, min_price: {2}".format(item_name, max_price, min_price)
    print(print_text)

item_name : 商品A, max_price: 100.0, min_price: 100.0
item_name : 商品S, max_price: 1900.0, min_price: 1900.0
item_name : 商品Z, max_price: 2600.0, min_price: 2600.0
item_name : 商品V, max_price: 2200.0, min_price: 2200.0
item_name : 商品O, max_price: 1500.0, min_price: 1500.0
item_name : 商品U, max_price: 2100.0, min_price: 2100.0
item_name : 商品L, max_price: 1200.0, min_price: 1200.0
item_name : 商品C, max_price: 300.0, min_price: 300.0
item_name : 商品I, max_price: 900.0, min_price: 900.0
item_name : 商品R, max_price: 1800.0, min_price: 1800.0
item_name : 商品X, max_price: 2400.0, min_price: 2400.0
item_name : 商品G, max_price: 700.0, min_price: 700.0
item_name : 商品P, max_price: 1600.0, min_price: 1600.0
item_name : 商品Q, max_price: 1700.0, min_price: 1700.0
item_name : 商品Y, max_price: 2500.0, min_price: 2500.0
item_name : 商品N, max_price: 1400.0, min_price: 1400.0
item_name : 商品W, max_price: 2300.0, min_price: 2300.0
item_name : 商品E, max_price: 500.0, min_price: 500.0
item_name : 商品K, max_price: 1100.0, mi

### ノック１６：顧客名の揺れを補正しよう

In [37]:
# まずはデータの揺れを確認する
user_df

# 顧客名に、半角全角スペースが混ざってる
# 登録日に日付と、数字データが混じってる

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127
...,...,...,...,...,...
195,川上 りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017/06/20
196,小松 季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018/06/20
197,白鳥 りえ,しらとり りえ,F市,shiratori_rie@example.com,2017/04/29
198,大西 隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019/04/19


In [38]:
# くっつける予定のデータと比較してみる
uriage_df

# user_dfが名前の間に空白が含まれているが、uriage_dfには存在しない

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904
...,...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500.0,福島友也,201902
2995,2019-06-22 04:03:43,商品M,1300.0,大倉晃司,201906
2996,2019-03-29 11:14:05,商品Q,1700.0,尾形小雁,201903
2997,2019-07-14 12:56:49,商品H,800.0,芦田博之,201907


In [39]:
# user_dfの顧客名から、空白を削除する
user_df['顧客名'] = user_df['顧客名'].str.replace(" ", "")
user_df['顧客名'] = user_df['顧客名'].str.replace("　", "")
user_df['顧客名']

0      須賀ひとみ
1       岡田敏也
2        芳賀希
3        荻野愛
4       栗田憲一
       ...  
195     川上りえ
196     小松季衣
197     白鳥りえ
198    大西隆之介
199     福井美希
Name: 顧客名, Length: 200, dtype: object

In [ ]:
# 今回は、空白削除だけだったが、顧客名単体での補正は現実的には難しい
# 同性同名の存在や誤変換などが考えられるので、ヒアリングや別資料をもらう必要性がある

### ノック１７：日付の揺れを補正しよう

In [40]:
user_df["登録日"]
# 数字の意味がわからないので解読する

# 当該データの形式がexcelなので、excelで見てみると正しく表示されているので、形式を変更する

0      2018/01/04
1           42782
2      2018/01/07
3           42872
4           43127
          ...    
195    2017/06/20
196    2018/06/20
197    2017/04/29
198    2019/04/19
199    2019/04/23
Name: 登録日, Length: 200, dtype: object

In [44]:
# まずは欠損データの数を見てみる. 数値データのみを選択して取り出す
flg_is_number = user_df["登録日"].astype("str").str.isdigit() # isdigit()で数値データのみを取得
flg_is_number.sum()

22

In [47]:
# https://qiita.com/y-vectorfield/items/323960a01d73ec1b4006
# excelの日付データを変換するときの注意点
# excelの日付データは1900/01/01を基準としているので、UNIXではないことを注意

# timedeltaで基準日からの経過時間をプラスする
conv_date = pd.to_timedelta(user_df.loc[flg_is_number, "登録日"].astype('float'), unit='D') + pd.to_datetime('1900/01/01')
conv_date

# excelとpythonでは日数の計算方法が違うので、注意。
# python側を-2日する

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 登録日, dtype: datetime64[ns]

In [49]:
# すべての日付データをdatetime化する
str_date = pd.to_datetime(user_df.loc[~flg_is_number, "登録日"])
str_date

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 登録日, Length: 178, dtype: datetime64[ns]

In [50]:
# 正しく置換したデータを結合する
user_df["登録日"] = pd.concat([conv_date, str_date]) # concatを保有しているindexが維持されるので大丈夫
user_df

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-18
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-19
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-29
...,...,...,...,...,...
195,川上りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017-06-20
196,小松季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018-06-20
197,白鳥りえ,しらとり りえ,F市,shiratori_rie@example.com,2017-04-29
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19


In [52]:
# 登録日を月ごとにまとめる
user_df['登録月'] = user_df['登録日'].dt.strftime("%Y%m")
user_df

,顧客名,かな,地域,メールアドレス,登録日,登録月
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,201801
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-18,201702
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07,201801
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-19,201705
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-29,201801
...,...,...,...,...,...,...
195,川上りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017-06-20,201706
196,小松季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018-06-20,201806
197,白鳥りえ,しらとり りえ,F市,shiratori_rie@example.com,2017-04-29,201704
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19,201904


In [56]:
# 登録月を集計してみてみる
temp = user_df.groupby("登録月").count()["顧客名"]
print(temp)
print(len(user_df))

登録月
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 顧客名, dtype: int64
200


### ノック１８：顧客名をキーに２つのデータを結合(ジョイン)しよう

In [58]:
# これまでの結果で、user_dfとuriage_dfでユーザ名が一致したので、結合してみる
merge_data = pd.merge(uriage_df, user_df, left_on="customer_name", right_on="顧客名", how="left")
merge_data = merge_data.drop("customer_name", axis=1) # ユーザ名が重複しているので削除する\
merge_data

,purchase_date,item_name,item_price,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録月
0,2019-06-13 18:02:34,商品A,100.0,201906,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,201701
1,2019-07-13 13:05:29,商品S,1900.0,201907,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,201804
2,2019-05-11 19:42:07,商品A,100.0,201905,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,201806
3,2019-02-12 23:40:45,商品Z,2600.0,201902,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,201807
4,2019-04-22 03:09:35,商品A,100.0,201904,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,201706
...,...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500.0,201902,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01,201707
2995,2019-06-22 04:03:43,商品M,1300.0,201906,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31,201803
2996,2019-03-29 11:14:05,商品Q,1700.0,201903,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15,201703
2997,2019-07-14 12:56:49,商品H,800.0,201907,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-15,201807


### ノック１９：クレンジングしたデータをダンプしよう

In [59]:
# データクレンジングして、キレイになったデータができたので、その状態で一時保存しておく
merge_data.to_csv("dump_data.csv", index=False) # indexをfalseにしないとデータが汚くなるので注意

### ノック２０：データを集計しよう

In [60]:
# 保存したデータから読み込む
import_data = pd.read_csv("dump_data.csv")
import_data

,purchase_date,item_name,item_price,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録月
0,2019-06-13 18:02:34,商品A,100.0,201906,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26 00:00:00,201701
1,2019-07-13 13:05:29,商品S,1900.0,201907,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07 00:00:00,201804
2,2019-05-11 19:42:07,商品A,100.0,201905,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19 00:00:00,201806
3,2019-02-12 23:40:45,商品Z,2600.0,201902,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22 00:00:00,201807
4,2019-04-22 03:09:35,商品A,100.0,201904,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07 00:00:00,201706
...,...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500.0,201902,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01 00:00:00,201707
2995,2019-06-22 04:03:43,商品M,1300.0,201906,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31 00:00:00,201803
2996,2019-03-29 11:14:05,商品Q,1700.0,201903,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15 00:00:00,201703
2997,2019-07-14 12:56:49,商品H,800.0,201907,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-15 00:00:00,201807


In [62]:
# 地域ごとの販売実績を見てみる
byRegion = import_data.pivot_table(index="purchase_month", columns='地域', aggfunc='size', fill_value=0)
byRegion

地域,A市,B市,C市,D市,E市,F市,G市,H市
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


In [63]:
# 集計期間で購入していないユーザを見てみる
# import_dataは購買履歴が基準になっているので、使えない。
# そこで、ユーザデータを基準としたデータを作成する
away_user_df = pd.merge(uriage_df, user_df, left_on='customer_name', right_on='顧客名', how='right')
away_user_df

,purchase_date,item_name,item_price,customer_name,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録月
0,2019-02-24 01:07:56,商品C,300.0,須賀ひとみ,201902,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,201801
1,2019-05-08 15:42:01,商品P,1600.0,須賀ひとみ,201905,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,201801
2,2019-07-03 07:49:05,商品M,1300.0,須賀ひとみ,201907,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,201801
3,2019-01-02 13:52:15,商品L,1200.0,須賀ひとみ,201901,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,201801
4,2019-06-29 04:58:36,商品R,1800.0,須賀ひとみ,201906,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,201801
...,...,...,...,...,...,...,...,...,...,...,...
2995,2019-02-17 06:40:12,商品S,1900.0,大西隆之介,201902,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19,201904
2996,2019-04-23 02:16:45,商品G,700.0,大西隆之介,201904,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19,201904
2997,2019-06-15 00:31:12,商品J,1000.0,大西隆之介,201906,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19,201904
2998,2019-07-17 23:00:47,商品K,1100.0,大西隆之介,201907,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19,201904


In [67]:
# 購入していないユーザがいれば、nanのデータが発生しているはず
away_user_df.isnull().sum()

purchase_date     1
item_name         1
item_price        1
customer_name     1
purchase_month    1
顧客名               0
かな                0
地域                0
メールアドレス           0
登録日               0
登録月               0
dtype: int64

In [69]:
# nanがあるので、購入してないユーザが存在する。
away_user_df[away_user_df["purchase_date"].isnull()]

,purchase_date,item_name,item_price,customer_name,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録月
2999,NaT,NaN,NaN,NaN,NaN,福井美希,ふくい みき,D市,fukui_miki1@example.com,2019-04-23,201904
